<a href="https://colab.research.google.com/github/choycoy/ednet/blob/master/ncf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 41.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [ ]:
!git clone https://github.com/microsoft/recommenders.git ./recommenders_microsoft

Cloning into './recommenders_microsoft'...
remote: Enumerating objects: 37381, done.
remote: Counting objects: 100% (913/913), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 37381 (delta 622), reused 762 (delta 582), pack-reused 36468
Receiving objects: 100% (37381/37381), 205.33 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (25266/25266), done.


In [ ]:
import sys
sys.path.append('/content/recommenders_microsoft')
sys.path.append('/content/recommenders_microsoft/recommenders')

In [ ]:
import sys
import os
import shutil
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Pandas version: 1.5.3
Tensorflow version: 2.11.0


In [ ]:
pip install scrapbook


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m


# Model parameters


SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "/content/recommenders_microsoft/recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/item_embeddings.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
columns_name=['userID','itemID','elapsed_time', 'timestamp']
df = pd.read_csv("/content/drive/My Drive/final/combined_with_filename_1.csv",sep=",",names=columns_name)
print('User Dataset 1 shape : {}'.format(df.shape))
print(df)

User Dataset 1 shape : (1000001, 4)
          userID       itemID  elapsed_time        timestamp
0        user_id  question_id  elapsed_time        timestamp
1              1         5012         38000       1.5651E+12
2              1         4706         24000       1.5651E+12
3              1         4366         68000       1.5651E+12
4              1         4829         42000       1.5651E+12
...          ...          ...           ...              ...
999996     10985        10927          1000  1571190000000.0
999997     10985        11948          1000  1571190000000.0
999998     10985        11949          1000  1571190000000.0
999999     10985        11947          1000  1571190000000.0
1000000    10985        11950          1000  1571190000000.0

[1000001 rows x 4 columns]


<ipython-input-26-32a837f4e8cb>:2: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/My Drive/final/combined_with_filename_1.csv",sep=",",names=columns_name)


In [ ]:
df = df.drop(df.index[0])
# Assuming df is your DataFrame
df = df.reset_index(drop=True)

In [ ]:
df

,userID,itemID,elapsed_time,timestamp
0,1,5012,38000,1.5651E+12
1,1,4706,24000,1.5651E+12
2,1,4366,68000,1.5651E+12
3,1,4829,42000,1.5651E+12
4,1,6528,59000,1.5651E+12
...,...,...,...,...
999995,10985,10927,1000,1571190000000.0
999996,10985,11948,1000,1571190000000.0
999997,10985,11949,1000,1571190000000.0
999998,10985,11947,1000,1571190000000.0


In [ ]:
# Read the CSV file and convert the elapsed_time column to numeric
df['elapsed_time'] = pd.to_numeric(df['elapsed_time'])

# Calculate quartiles of elapsed time
quartiles = df['elapsed_time'].quantile([0.2, 0.4, 0.6, 0.8])

# Define rating_bins based on quartiles and add 'inf' for values greater than the 80th percentile
rating_bins = [0] + quartiles.tolist() + [float('inf')]

# Create corresponding ratings based on the number of bins (5-point rating scale)
ratings = [1, 2, 3, 4, 5]

# Create a new column for ratings
df['rating'] = pd.cut(df['elapsed_time'], bins=rating_bins, labels=ratings, include_lowest=True)

df.drop('elapsed_time', axis=1, inplace=True)
# Print the dataframe with the new 'rating' column
print(df)

       userID itemID        timestamp rating
0           1   5012       1.5651E+12      5
1           1   4706       1.5651E+12      3
2           1   4366       1.5651E+12      5
3           1   4829       1.5651E+12      5
4           1   6528       1.5651E+12      5
...       ...    ...              ...    ...
999995  10985  10927  1571190000000.0      1
999996  10985  11948  1571190000000.0      1
999997  10985  11949  1571190000000.0      1
999998  10985  11947  1571190000000.0      1
999999  10985  11950  1571190000000.0      1

[1000000 rows x 4 columns]


In [ ]:
df['rating'] = df['rating'].astype(float)
df['timestamp'] = df['timestamp'].astype(float)
df['userID'] = df['userID'].astype(int)
df['itemID'] = df['itemID'].astype(int)

In [ ]:
train, test = python_chrono_split(df, ratio=0.75)

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [ ]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [ ]:
train_file = "./train.csv"
test_file = "./test.csv"
leave_one_out_test_file = "./leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [ ]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

100%|██████████| 854/854 [00:11<00:00, 71.79it/s]


In [ ]:
EPOCHS = 1
BATCH_SIZE =1024

In [ ]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 37.159790809000015 seconds for training.


In [ ]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,1.0,6766.0,0.490251
1,1.0,6768.0,0.611744
2,1.0,6765.0,0.677622
3,1.0,6767.0,0.627829
4,1.0,5611.0,0.422656


In [ ]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 120.24502622500017 seconds for prediction.


In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.009331
NDCG:	0.278991
Precision@K:	0.199297
Recall@K:	0.017951
